# Magnitudes

We commonly describe an earthquakes size based on it's magnitude. There are various magnitude scales in use, but all
vary logarithmicly. In a general sense, the energy released by a magnitude 2 earthquake is ten times greater than a 
magnitude 1. However, most magnitude scales are not perfectly related to energy, only the moment magnitude ($M_W$) scale
is tied directly to energy release.

In this notebook we will look at how the following magnitudes are calculated:
- Local Magnitude ($M_L$;
- Coda-duration Magnitude ($M_D$);
- Moment Magnitude ($M_W$);
- W-phase derived Moment Magnitude ($M_{WW}$).

The theory behind most of this can be found in Section 4.6 of Stein and Wysession. For more information on the W-phase
moment magnitude method (which is now routinely in use by USGS for fast determination of large earthquake magnitudes),
see [Duputel et al., 2012](https://academic.oup.com/gji/article/189/2/1125/621962).

For a list of other commonly used scales, see the [USGS table of magnitude.](https://earthquake.usgs.gov/learn/topics/mag-intensity/magnitude-types.php)

## Local Magnitude

$M_L$ began life as the *Richter magnitude*, however, this is a magnitude scale with specific scaling parameters for Southern
California. The same equation form used in the initial Richter scale is used elsewhere, but with appropriate attenuation
parameters: thus, if anyone is caught discussing how large earthquakes are in New Zealand *on the Richter scale* they will
be removed from the class!

$M_L$ is based on the measurement of the peak amplitude in a waveform recorded by a *Wood-Anderson* seismometer. There are
very few Wood-Anderson seismometers still in operation today, but we know what the instrument response of a Wood-Anderson
instrument is, and can therefore simulate Wood-Anderson waveforms from other instruments.  The original Richter local magnitude
scale is:

\begin{equation*}
    M_L = \log{A} + 2.76\log{\Delta} - 2.48
\end{equation*}
where $A$ is the peak amplitude on a Wood-Anderson seismometer, $\Delta$ is the epicentral distance. This only works
well for shallow earthquakes. The second term effectively corrects for geometrical spreading, while the final factor
is included to scale the magnitude.

Recent work by [Michailos et al., 2019](https://agupubs.onlinelibrary.wiley.com/doi/pdf/10.1029/2018GC007743) derived
a robust local magnitude scale for the Southern Alps tied to moment magnitude:

\begin{equation*}
    M_L = \log{A} + \log{r} + \eta r - S - 3.644
\end{equation*}
where $r$ is the hypocentral distance, $\eta$ is an attenutaion parameter, which, for $r\le 60km$ is $1.2\times 10^{-2}km^{-1}$
and for $r>60km$ is $0.01\times 10^{-2}km{-1}$. $S$ is a site dependent correction which incorporate local site effects. Geometrical
spreading is assumed to be logarthimically related to $r$, with a gradient of 1.

Lets look at how we would pick an amplitude for local magnitude for a nearby earthquake:

## Coda-duration Magnitude

## Moment Magnitude

## W-phase Moment Magnitude